In [2]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
%matplotlib inline
#%matplotlib notebook

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
home directory is /home/heineike/
y1000plus_tools dir is /home/heineike/github/y1000plus_tools/
y1000plus data dir is /home/heineike/genomes/y1000plus/
Added /home/heineike/github/y1000plus_tools/ to path
Importing y1000plus_tools.py
home directory is 
y1000plus_tools dir is 
y1000plus data dir is 
Added /home/heineike/github/expression_broad_data/core/ to path
importing io_library.py
Importing io_library.  If autoreload, may need to reset base_dir and data_processing dir 
  io_library.base_dir=base_dir 
 io_library.data_processing_dir = data_processing_dir
sys.path : 

['/home/heineike/github/y1000plus_tools/scripts', '', '/home/lab/envs/seqanalysis/lib/python35.zip', '/home/lab/envs/seqanalysis/lib/python3.5', '/home/lab/envs/seqanalysis/lib/python3.5/plat-linux', '/home/lab/envs/seqanalysis/lib/python3.5/lib-dynload', '/home/lab/envs/seqanalysis/lib/python3.5/site-packages', '/home/lab/envs/seq

In [12]:
y1000_species = pd.read_csv(y1000plus_dir + "y1000_species_table.csv", index_col=0)

#not including outgroup species
#y1000_species_subset = y1000_species
y1000_species_subset = y1000_species[y1000_species['Major clade']!='outgroup']

# #Select only clades of interest: 
# clades_of_interest = {'Saccharomycetaceae','Saccharomycodaceae','Phaffomycetaceae','CUG-Ser2','CUG-Ser1','Pichiaceae','CUG-Ala'}
# #clades_of_interest = {'Saccharomycetaceae'}
# y1000_species_subset = y1000_species[y1000_species['Major clade'].isin(clades_of_interest)]

#outgroup = 'Lipomyces starkeyi'
#outgroup_orig_genome = y1000_species[y1000_species['Species name']==outgroup]['original_genome_id'].values[0]
#'Yarrowia lipolytica'
# #outgroup = 'Hanseniaspora vineae'  #For EGO2/4 - only Sacch so use different outgroup
# y1000_species_subset = pd.concat([y1000_species_subset, y1000_species[y1000_species['Species name']==outgroup]])


In [13]:

gois = ['YKR054C']

#Make promoter files for various genes of interest
#Load S.Cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/saccharomyces_cerevisiae.csv')

scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
# target_y1000_id = scer_lookup.loc[goi,'y1000_id']

y1000_ids = dict(scer_lookup.loc[gois,'y1000_id'])
y1000_id_list = [y1000_ids[goi] for goi in gois]

goi_og_lookup, og_genes_lookup = y1000plus_tools.make_og_genes_lookup(y1000_id_list, y1000_species_subset)

In [17]:
goi_og_lookup

{'110_1706': 'OG2473'}

In [14]:
goi_common_name = 'DYN1'
y1000_id = '110_1706'
og = goi_og_lookup[y1000_id]
og_genes = og_genes_lookup[og]
protein_fname = goi_common_name + '_' + og
y1000plus_tools.extract_protein_seqs(og_genes, protein_fname, y1000_species_subset)
print(goi_common_name + ' protein sequence extraction complete')

komagataella_pastoris
metschnikowia_similis
saccharomyces_cerevisiae
yHMPu5000034974_deakozyma_indianensis_160519
yarrowia_lipolytica
yHMPu5000034624_pichia_nakasei_160519
zygosaccharomyces_rouxii
babjeviella_inositovora
yHAB159_kazachstania_solicola_160519
yHMPu5000034632_candida_athensensis_160519
yHMPu5000041713_debaryomyces_maramus_160928
candida_dubliniensis
nakazawaea_peltata
spathaspora_girioi
yHMPu5000041862_candida_golubevii_160928
metschnikowia_arizonensis
metschnikowia_bicuspidata
yHMPu5000034612_saturnispora_saitoi_160519
metschnikowia_continentalis
yHMPu5000034748_lipomyces_oligophaga_160519
yHMPu5000034660_diddensiella_caesifluorescens_160519
yHMPu5000034988_candida_fructus_160519
yHAB166_kazachstania_yakushimaensis_160519
yHMPu5000035047_barnettozyma_pratensis_160519
lachancea_fermentati
yHMPu5000035673_candida_orba_160613
lodderomyces_elongisporus
candida_infanticola
yHMPu5000034892_ogataea_pilisensis_160519
yHMPu5000034673_blastobotrys_nivea_160519
metschnikowia_santac

In [15]:
outgroup = 'Yarrowia lipolytica'#'Lipomyces starkeyi'
outgroup_orig_genome = y1000_species[y1000_species['Species name']==outgroup]['original_genome_id'].values[0]


In [16]:
# Mafft command 

#Make tree: 

#Use Mafft to construct an MSA

#goi_common_name and og already set above

protein_fname_base = y1000plus_dir + os.path.normpath('proteins_og/' + goi_common_name + '_' + og)


#Command: 
mafft_command = ['mafft', '--genafpair', '--maxiterate', '1000', 
                protein_fname_base + '.fasta', 
                '>',
                protein_fname_base + '_aln.fasta']

#Running it from this notebook seems slow
#!{" ".join(mafft_command)}

print(" ".join(mafft_command))

#This is the E-ENS-i algorithm which is suitable for conserved regions separated by long unaligned gaps. https://mafft.cbrc.jp/alignment/software/algorithms/algorithms.html
#tree command: (designated outgroup)

#
outgroup_gene = None
prot_seqs = SeqIO.parse(protein_fname_base + '.fasta', "fasta")
for seq in prot_seqs: 
    if seq.id.split('Seq')[0]== outgroup_orig_genome+'_':
    #if seq.id.split('Seq')[0]=='hanseniaspora_vinae_':
        outgroup_gene = seq.id
        
if outgroup_gene == None: 
    print('No yarrowia_lipolytica ortholog - need to pick a different outgroup')

    
    
#Note: for iqtree to run you need to activate the seqanalysis python environment: 
#. activate /home/lab/envs/seqanalysis

iqtree_command = ["iqtree", 
                  "-s" , protein_fname_base + '_aln_trimmed.fasta',
                  "-nt", "AUTO",  #automatically determines number of threads 
                  "-o", outgroup_gene]
print(" ".join(iqtree_command))

#Current trimming protocol - open up on Jalview and trim by eye. Save as _trimmed.fasta, then find and replace
# all "/" at the end of each gene name with a space. 


# #Use the MSA to build a tree

# #from that tree for all post WGD species, 
#     #see if gene is closer to "low" or "high" goi.  Assign pre_wgh, low, or high label
#     #could verify that outgroup doesn't appear between target and S.Cer low or high gene as well. 
    
# #Order search results based on the tree

mafft --genafpair --maxiterate 1000 /home/heineike/genomes/y1000plus/proteins_og/DYN1_OG2473.fasta > /home/heineike/genomes/y1000plus/proteins_og/DYN1_OG2473_aln.fasta
iqtree -s /home/heineike/genomes/y1000plus/proteins_og/DYN1_OG2473_aln_trimmed.fasta -nt AUTO -o yarrowia_lipolytica_Seq_1861
